In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
schema_name = 'artemis'

In [0]:
spark.sql(f"use {schema_name}")

In [0]:
display(
    spark.sql(f'select * from {schema_name}.training_data where kwh > 4.6')
)

In [0]:
df = (
    spark
    .read
    .table(f'{schema_name}.training_data')
    .drop('mac', 'serial', 'name', 'client_id', 'client_name', 'pickaxe_id', 'group_id', 'rack_id', 'hashing_uptime_low', 'hashing_uptime_normal', 'hashing_uptime_high')
).dropna()
display(df)

In [0]:
min_date = df.agg({"time": "min"}).collect()[0][0]
max_date = df.agg({"time": "max"}).collect()[0][0]
print(f"{min_date} | {max_date}")

In [0]:
train_df = df.filter(df.time <= '2025-04-01')
val_df = df.filter(df.time > '2025-04-01')
display(train_df)

In [0]:
# Indexing categorical variables
indexer = StringIndexer(
    inputCols=["type", "row", "index"],
    outputCols=["type_index", "row_index", "index_index"]
)
feature_table = indexer.fit(train_df).transform(train_df)

# One-hot encoding for categorical variables
encoder = OneHotEncoder(
    inputCols=["type_index", "row_index", "index_index"],
    outputCols=["type_ohe", "row_ohe", "index_ohe"]
)
ohe_model = encoder.fit(feature_table)

cat_df = ohe_model.transform(feature_table)

display(cat_df)

In [0]:
# Define the window specification
window_spec = Window.partitionBy("miner_id").orderBy("time")

In [0]:
# Handle NULL values by filling them with a default value (e.g., 0)
cat_df = cat_df.na.fill({"kwh": 0, "hash_rate": 0})

# Calculate trailing averages for the specified time windows
trailing_trends_df = (
    cat_df
    .withColumn("kwh_avg_3h", F.avg("kwh").over(window_spec.rowsBetween(-3*60, 0)))
    .withColumn("kwh_avg_6h", F.avg("kwh").over(window_spec.rowsBetween(-6*60, 0)))
    .withColumn("kwh_avg_9h", F.avg("kwh").over(window_spec.rowsBetween(-9*60, 0)))
    .withColumn("kwh_avg_12h", F.avg("kwh").over(window_spec.rowsBetween(-12*60, 0)))
    .withColumn("kwh_avg_24h", F.avg("kwh").over(window_spec.rowsBetween(-24*60, 0)))
    .withColumn("hash_rate_avg_3h", F.avg("hash_rate").over(window_spec.rowsBetween(-3*60, 0)))
    .withColumn("hash_rate_avg_6h", F.avg("hash_rate").over(window_spec.rowsBetween(-6*60, 0)))
    .withColumn("hash_rate_avg_9h", F.avg("hash_rate").over(window_spec.rowsBetween(-9*60, 0)))
    .withColumn("hash_rate_avg_12h", F.avg("hash_rate").over(window_spec.rowsBetween(-12*60, 0)))
    .withColumn("hash_rate_avg_24h", F.avg("hash_rate").over(window_spec.rowsBetween(-24*60, 0)))
)

display(trailing_trends_df)

In [0]:
# Create a label column based on the condition
labeled_trends_df = (
    trailing_trends_df
    .withColumn("label", F.when(F.avg("kwh").over(window_spec.rowsBetween(0, 4*60)) >= 4.6, True).otherwise(False))
)

display(labeled_trends_df)